# Building Scalable Retrieval System With Two-Tower Models

This project aims to create a recommendation system for an online store that can quickly find the most fitting products when a user types in a search query. The focus is on using the Two Tower Search Query Retrieval Model, a deep learning architecture designed for tasks like helping users search for items or suggesting products.

The Two Tower model has two parts – one dedicated to handling the search query (**`Query tower`**) and the other to manage the products (**`Item tower`**). For this project, I will use a dataset based on *Amazon Shopping Queries*, including complex search queries and judgments on how relevant the products are to those queries.

To make this recommendation system effective, I will incorporate pre-existing word-match embeddings and information gathered from past shopping behaviors, like what users previously bought or looked at. Additionally, I will fine-tune these embeddings by adding other signals such as category, price, ratings, and reviews. This will help the system to learn and improve its recommendations based on past user interactions, even personalizing suggestions for different user groups.

### Two Tower model

The two-tower (or twin-tower) embedding model is a model training method for connecting embeddings in two different modalities by placing both modalities in the same vector space. For example, a two-tower model could generate embeddings of both images and text in the same vector space. Personalized recommendation systems often use items and user-histories as the two different modalities. The modalities need to be “grounded”. For example, image and text can be “grounded” by creating training data where a caption matches an image. Two-tower models are able to map embeddings from different modalities into the same space by ensuring both modalities have the same dimension “d”. For example, if the item embedding is of length 100, then the query embedding dimension should be 100.

![Alt Text](image.png)

https://www.hopsworks.ai/dictionary/two-tower-embedding-model

# Search Relevance

The notion of binary relevance limits the customer experience. Hence, we break down relevance into four classes (ESCI) which are used to measure the relevance of the items in the search results:
- **Exact (E):** the item is relevant for the query, and satisfies all the query specifications (e.g., water bottle matching all attributes of a query “plastic water bottle 24oz”, such as material and size)
- **Substitute (S):** the item is somewhat relevant: it fails to fulfill some aspects of the query but the item can be used as a functional substitute (e.g., fleece for a “sweater” query)
- **Complement (C):** the item does not fulfill the query, but could be used in combination with an exact item (e.g., track pants for “running shoe” query)
- **Irrelevant (I):** the item is irrelevant, or it fails to fulfill a central aspect of the query (e.g. socks for a “pant” query)

# Shopping Queries Dataset

We introduced the “Shopping Queries Data Set”, a large dataset of difficult search queries, published with aim of fostering research in the area of query-product semantic matching.
- Massive dataset with 2.6 million human annotations (ImageNet for Product search).
- Some important Characteristics of this dataset:
- It is derived from real customers searching for real products online. Products are linked to online catalog.
- For each query, the dataset provides a list of up to 40 potentially relevant results, together with ESCI relevance judgements.
- The dataset is Multilingual, as it contains queries in English, Japanese, and Spanish. It provides both breadth (a large number of queries) and depth (≈20 results per query), unlike other public datasets.
- All results have been manually labeled with multi-valued relevance labels in the context of e-shopping.
- Queries are not randomly sampled, but rather, subsets of the queries have been sampled specifically to
provide a variety of challenging problems (such as negation, attribute parsing, etc.).
- Each query-product pair is accompanied by some additional public catalog information (including title, product description, and additional product related bullet points). This has both categorical and textual metadata, and multiple levels of representation (from a short title to a long description of the product).

Query Parsing aims to extract product attributes, such as color, brand, and product type, from search queries. These attributes play a crucial role in search engines for tasks such as matching, ranking, and recommendation. T

In [1]:
import pandas as pd # data processing, CSV file I/O
import numpy as np
from annoy import AnnoyIndex

In [2]:
data = pd.read_csv('dataset_150k.csv')
data.head()

,query,product_id,esci_label,split,q0,q1,q2,q3,q4,q5,...,p150,p151,p152,p153,p154,p155,p156,p157,p158,p159
0,monogram crossbody bags for women,B07SC67L97,E,train,0.007149,-0.036234,-0.021769,0.071798,0.004489,0.046100,...,-0.086586,0.009924,0.015074,0.014845,0.030969,-0.002622,-0.004156,-0.004414,-0.052931,-0.062627
1,monogram crossbody bags for women,B088BMKTVZ,E,train,0.007149,-0.036234,-0.021769,0.071798,0.004489,0.046100,...,-0.013865,0.022348,0.072437,0.017069,0.044085,0.034383,-0.037070,-0.031683,-0.046816,-0.130665
2,monogram crossbody bags for women,B07NVTDGVN,E,train,0.007149,-0.036234,-0.021769,0.071798,0.004489,0.046100,...,-0.034130,-0.007226,0.030359,0.017682,0.054370,-0.040309,-0.028475,0.002260,-0.008909,-0.113362
3,replicia tory burch bags,B07NVTDGVN,E,train,-0.010582,-0.032959,-0.027428,0.031883,-0.000479,0.062785,...,-0.034130,-0.007226,0.030359,0.017682,0.054370,-0.040309,-0.028475,0.002260,-0.008909,-0.113362
4,green advertine crystal,B07H8GJ44Q,E,train,0.025034,-0.025958,-0.021118,0.026299,0.006780,0.022166,...,-0.053963,0.011845,0.031754,0.026041,0.061598,-0.041917,-0.050758,0.015715,-0.050046,-0.078058


In [3]:
query = pd.read_csv('query_150k.csv')
query.head()

,q0,q1,q2,q3,q4,q5,q6,q7,q8,q9,...,q23,q24,q25,q26,q27,q28,q29,q30,q31,query
0,0.016756,-0.072765,0.010259,0.048307,0.074665,0.084850,0.022835,-0.066671,0.039883,-0.027148,...,-0.002145,0.011363,0.015675,0.018649,0.025668,0.000701,-0.006060,-0.078536,-0.026082,child proof cabinet locks
1,-0.007341,-0.010955,0.059960,0.071282,0.022258,0.032136,0.042009,-0.041242,-0.009459,-0.017514,...,-0.009223,-0.014555,-0.002024,0.062305,0.020665,-0.005689,0.032699,-0.045051,-0.096923,ankle stockings for women sheer
2,0.027613,0.019582,-0.032154,0.018755,0.037383,-0.025616,0.024898,-0.025434,-0.047236,0.022970,...,0.019245,-0.007898,-0.011111,0.059078,0.020216,0.017000,-0.034270,-0.007410,-0.026015,gluten free snacks
3,0.021799,-0.018171,0.005133,0.049379,0.021239,-0.004974,0.022700,-0.024240,-0.014328,0.018335,...,0.033078,0.003084,-0.041141,0.033879,-0.002729,-0.015864,0.026388,-0.021622,-0.083685,hair geow
4,0.036940,-0.077912,-0.036468,0.036546,0.121124,0.009458,0.050458,-0.060216,-0.033329,-0.018182,...,0.010799,0.049342,0.031401,0.065255,0.086359,-0.056535,-0.013196,-0.023676,-0.070366,"1 by one, amplified, outdoor hdtv antenna"


In [4]:
product2 = pd.read_csv('product_150k.csv')
product2.head()

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p151,p152,p153,p154,p155,p156,p157,p158,p159,product_id
0,0.057728,0.011255,-0.023404,0.092429,-0.020029,0.018735,0.003974,0.008556,-0.005657,0.016547,...,-0.021550,0.006544,0.006825,0.076608,-0.019186,-0.010564,0.000239,-0.043817,-0.097729,B01A5DU2TA
1,0.038126,0.004931,-0.022067,0.066682,0.009379,0.017764,0.019343,0.039371,0.001445,0.010448,...,0.004315,0.048992,0.021710,0.093479,-0.026546,-0.052332,0.004328,-0.019683,-0.112607,B07346RPKN
2,0.036354,0.016651,0.012105,0.096476,0.024649,-0.013194,0.018663,0.024827,-0.021811,-0.019258,...,0.017124,0.017427,0.026384,0.075618,0.015224,-0.049442,0.055570,-0.058420,-0.071863,B078TCVZ4V
3,0.046325,0.016477,-0.022943,0.018492,-0.025954,-0.005175,0.018342,0.041428,-0.006838,-0.008621,...,0.008416,0.010578,0.021611,0.051968,-0.002022,-0.055210,0.022411,-0.055057,-0.084781,B07BR3PPDR
4,0.042861,-0.012059,-0.051115,0.127641,0.026230,0.011335,0.004991,0.008443,0.037345,-0.019857,...,0.008416,0.010578,0.021611,0.051968,-0.002022,-0.055210,0.022411,-0.055057,-0.084781,B07H5C4P19


In [5]:
product = pd.read_parquet('shopping_queries_dataset_products.parquet')
product.head()

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale
0,B079VKKJN7,"11 Degrees de los Hombres Playera con Logo, Ne...",Esta playera con el logo de la marca Carrier d...,11 Degrees Negro Playera con logo\nA estrenar ...,11 Degrees,Negro,es
1,B079Y9VRKS,Camiseta Eleven Degrees Core TS White (M),None,None,11 Degrees,Blanco,es
2,B07DP4LM9H,11 Degrees de los Hombres Core Pull Over Hoodi...,La sudadera con capucha Core Pull Over de 11 G...,11 Degrees Azul Core Pull Over Hoodie\nA estre...,11 Degrees,Azul,es
3,B07G37B9HP,11 Degrees Poli Panel Track Pant XL Black,None,None,11 Degrees,None,es
4,B07LCTGDHY,11 Degrees Gorra Trucker Negro OSFA (Talla úni...,None,None,11 Degrees,Negro (,es


This means that the product tower model encodes different aspects of a product across 5 different subspaces.
Each of the 5 subspaces represent different features, such as:
- product_description
- product_bullet_point
- product_brand
- product_color
- product_locale

### Merge datasets

In [6]:
product_embedding = pd.merge(
    product2, 
    product[['product_id', 'product_title']].drop_duplicates(), 
    on='product_id'
).reset_index(drop=True)
product_embedding.head()

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p152,p153,p154,p155,p156,p157,p158,p159,product_id,product_title
0,0.057728,0.011255,-0.023404,0.092429,-0.020029,0.018735,0.003974,0.008556,-0.005657,0.016547,...,0.006544,0.006825,0.076608,-0.019186,-0.010564,0.000239,-0.043817,-0.097729,B01A5DU2TA,"Crocs Kids' Handle It Rain Boots , Candy Pink,..."
1,0.038126,0.004931,-0.022067,0.066682,0.009379,0.017764,0.019343,0.039371,0.001445,0.010448,...,0.048992,0.021710,0.093479,-0.026546,-0.052332,0.004328,-0.019683,-0.112607,B07346RPKN,"Hatley Kids' Little Classic Rain Boots, Pink &..."
2,0.036354,0.016651,0.012105,0.096476,0.024649,-0.013194,0.018663,0.024827,-0.021811,-0.019258,...,0.017427,0.026384,0.075618,0.015224,-0.049442,0.055570,-0.058420,-0.071863,B078TCVZ4V,Outee Rubber Kids Rain Boots
3,0.046325,0.016477,-0.022943,0.018492,-0.025954,-0.005175,0.018342,0.041428,-0.006838,-0.008621,...,0.010578,0.021611,0.051968,-0.002022,-0.055210,0.022411,-0.055057,-0.084781,B07BR3PPDR,Hope & Henry Girls' Red Milano Stitch Cardigan
4,0.042861,-0.012059,-0.051115,0.127641,0.026230,0.011335,0.004991,0.008443,0.037345,-0.019857,...,0.010578,0.021611,0.051968,-0.002022,-0.055210,0.022411,-0.055057,-0.084781,B07H5C4P19,Spring&Gege Youth Solid Full Zipper Hoodies So...


In [7]:
product_embedding['prod_id'] = range(0, product_embedding.shape[0])

In [8]:
query['que_id'] = range(0, query.shape[0])

Before training the model, I'll check the quality of dataset and input embeddings. ANN search will be run for queries and ANN search for products to see whether semantic/text closeness is captured in the embedddings or not.

<br>

# Approximate Nearest Neighbor (ANN) Search: The Foundation of AI-Powered Search Technology  

## What is ANN Search?  
Approximate Nearest Neighbor (**ANN**) search is a technique used in **vector databases** to find data points closest to a given query point with a certain level of approximation. Unlike traditional search methods, **ANN search is optimized for efficiency**, especially in **high-dimensional spaces**.

### **Key Features of ANN Search**
- Finds data points **close to a query** while balancing accuracy and computational feasibility.
- Excels in **handling large and complex datasets**.
- Frequently used in AI applications such as:
  - **Music streaming**: Recommending songs similar to user preferences.
  - **Medical imaging**: Identifying similar diagnostic images for quicker diagnoses.
  - **Image recognition**: Rapidly finding visually similar images.

## **How ANN Search Works**  
ANN search operates on the principle of finding **data points closest to a query** while prioritizing speed over exactness.

### **Key Concepts of ANN Search**
- **Uses efficient algorithms** like:
  - **Locality-Sensitive Hashing (LSH)** → Places similar items into the same bucket to reduce search time.
  - **Graph-Based Methods** → Treats data points as nodes in a graph to improve search speed.
- **Avoids exhaustive searching** by narrowing down search areas.
- **Handles challenges** like avoiding local minimums by exploring multiple paths.

### **Comparison with Exact Nearest Neighbor (NN) Search**
| Feature  | ANN Search  | NN Search (Exact) |
|----------|------------|------------------|
| **Accuracy**  | Approximate  | Exact |
| **Speed**  | Fast | Slower for large datasets |
| **Scalability** | Works well in high-dimensional data | Struggles in high-dimensional data |
| **Use Cases** | AI, recommendations, large-scale applications | Small datasets, low-dimensional analysis |

## **Advanced Techniques in ANN Search**  
ANN search benefits from modern **optimization techniques** to improve performance:

### **1. Distance Metrics Used**
- **Euclidean Distance** → For numerical vectors.
- **Manhattan Distance** → When feature values vary in scale.
- **Hamming Distance** → For binary data comparisons.

### **2. Dimensionality Reduction**
To improve efficiency, ANN search often uses:
- **Principal Component Analysis (PCA)** → Reduces dimensions while preserving key information.
- **t-Distributed Stochastic Neighbor Embedding (t-SNE)** → Helps visualize and cluster high-dimensional data.

## **The Role of ANN Search in Vector Databases**  
Vector databases store data as **high-dimensional vectors** rather than traditional row-column formats. ANN search is particularly useful in:
- **AI-driven search applications** (e.g., Google Lens, reverse image search).
- **Fast similarity search** by computing vector distances.
- **Efficient large-scale data retrieval**.

In [9]:
query_tower_dimensions = 32
product_tower_dimensions = 160

Dot product between query and product embeddings is not possible as both are in different dimensions.

Using AnnoyIndex from annoy package for fast approximate nearest neighbor (ANN) search.

# Query Tower

### 1. Why Use Annoy?

Problem: Searching Similar Vectors is Computationally Expensive
Imagine you have 70,000 queries and 150,000 products, each represented as a 32-dimensional or 160-dimensional vector.
A simple brute-force approach to find the nearest neighbors for a query would require computing distances against all other queries/products, which is computationally expensive.

💡 Annoy solves this by using an efficient indexing structure (Random Projection Trees) to speed up nearest neighbor search.

Steps:

1. Use Annoy to build a fast search index - Avoid slow brute-force search
2. Store query embeddings in Annoy - Allow quick lookup of similar queries
3. Find k-nearest neighbors	- Retrieve the most similar queries
4. Store results in a DataFrame	- Easy visualization and analysis

In [10]:
# Create an Annoy index for the query embeddings (using Euclidean distance)
index = AnnoyIndex(query_tower_dimensions, 'euclidean')

# Dictionary to map query id to query text
map_query = {}

for ix,row in query.iterrows():
    map_query[row['que_id']] = row['query']
    
    key = int(row['que_id'])
    vec = list(row[['q' +str(x) for x in list(range(query_tower_dimensions))]])
    index.add_item(key,vec)

Annoy creates a searchable structure using 100 trees.

In [11]:
index.build(100) # 100 trees
index.save('query.tree')

# This allows fast lookup without recomputing embeddings.
index = AnnoyIndex(query_tower_dimensions,  'euclidean')
index.load('query.tree')

True

In [12]:
# Find nearest neighbors for each query
top_k = 20 # Get the top 20 nearest neighbors
results = []

# Loop through each query and find its nearest neighbors
# For each query, Annoy quickly finds the top k=20 nearest neighbors.
for ix,row in query.iterrows():
    item = row['query']
    results.append([item] + [map_query[x] for x in index.get_nns_by_item(row['que_id'], top_k+1)[1:]])
    
    if ix == 50:
        break
    
# Define column names for the resulting DataFrame
cols = ['query_id']
for i in range(top_k):
    cols += ['nearest_{}'.format(i+1)]

# Create the results DataFrame
neighbors = pd.DataFrame(results, columns = cols)

display(neighbors.head(50))

,query_id,nearest_1,nearest_2,nearest_3,nearest_4,nearest_5,nearest_6,nearest_7,nearest_8,nearest_9,...,nearest_11,nearest_12,nearest_13,nearest_14,nearest_15,nearest_16,nearest_17,nearest_18,nearest_19,nearest_20
0,child proof cabinet locks,child locks for cabinets,cabinet locks,cabinet handles,magnetic child safety cabinet locks,lockable tool bag,ring cleaning pen,kids safety locks for cabinets,latch hook kits,electromagnetic door lock,...,surelock luggage lock,infant floor pad,baby safety locks for cabinets,baby proofing cabinet lock bows,cabinet lock,childrens kitchen set,door child lock,kitchen sink hose attachment,gingham door mat,shower garden hose wand
1,ankle stockings for women sheer,cargo pants for men,short cowboy booties for women,pants for women,dress boots for men,waist chains for women,pencil skirts for women,socks long for women,work boots for men,striped panties for women,...,underwear for men,cat boots for men,dropcloths,cargo pants for women,back slit shirts for women,striped dresses for women,slippers for men,day pants for men,socks for booties women,skirts for women
2,gluten free snacks,gluten free lemon cookie,heartland gluten free pasta,vegan jerky,gluten free bread,cereals sugar free,beneful wet dog food,chewy granola bars,lays hot pickle,pickled food,...,chocolate covered raisins,smart tart,wrinkle reducing cream,jerky without sugar,gluten free stuffing mix,beef jerky,miltons gluten free crackers,yung gravy,bleaching cream,peppermint tea bags
3,hair geow,arhletic sweatshirts,pink earrinfs,snoogle,mypillow sheets,sheepjes yarn,furinno,ziploc,dr browns nipples,stihl bar,...,gillie suit,calligrpahy,crocs,maraschino cherries,lacross boots men rubber,snapple,kill suit gillie,blue wif,dipwell,timberlans
4,"1 by one, amplified, outdoor hdtv antenna",lego 14 years and up,1/3 amber mist bottles without fist mist sprayers,1 ear wired headphones without mic,lume deodoant for private parts,avery two tone dividers,1.25 drain popper without overflow,over the ear headphones charging dock,work headphones,bar set without shaker,...,whistles without lanyard,stand for headphones,11. he y treverse matts,unplugged wedding sign,now or laters candy,"laser, self-leveling, 3-beam",(hearing aid not amplifer),wired noise cancelling headphones without deep...,clinique happy for men,duralex glasses set
5,adidas sweats slim,fila sweatpants,galaxy adidas shoes men,adidas pants men slim,adidas soccer cleats black,adidas pants men,adidas soccer cleats,adidas superstar all white,adidas beanie,adidas pants,...,adidas sweatband,mens russell sweatpants,adidas running shoes mens,billie eilish hoodie,adidas shoes men,running jacket women adidas,adidas original superstar women,adidas mens shoes samba,adidas pants women,neck warmer adidas
6,beta sitosterol now,help the beatles mps,hempcrete,liw hisyamine probiotic,zervos,dr dans cortibalm lip balm,amzcool,crystal bags to hold crystals,graco modes to grow,altavoz jbl,...,cool wallet gocci collors,didlo vibarator sex,credit card tool wallet multitool,lansinoh therapearl 3-in-1 hot or cold breast ...,bathing suit surong women,testalize,zometool,kangol,kiss i gloo,methlypro
7,iphone 11 pro,ipad pro 11,ipad pro,screen protector iphone 8,apple ipad pro 11,apple iphone 11 pro unlocked,screen protector iphone 6s,iphone 11 accessories,ipad pro screen protector 11,screen protector iphone x,...,apple iphone 8 case,phone case iphone 8,iphone 11 pro max,phone screen protector iphone 7,iphone xr cases,apple ipad pro,phone case iphone 6,wireless controller ipad pro,iphone 6s,apple iphone 7 case
8,large training pads for dogs,dog beds for large dogs,dog costumes for large dogs,pet beds for large dogs,dog harness for large dogs,water bowls for large dogs,dog clothes for large dogs,mats for exercise,extra large dog beds for german shepherds,shock collar for 2 dogs,...,food bowls for small dogs,dog belly bands for male dogs,camera for monitoring dog,250 lb resistance bands for men,harness for small dogs,nerf f

# Product Tower

In [13]:
product_index = AnnoyIndex(product_tower_dimensions, 'euclidean')
map_product_dict = {}

for ix,row in product_embedding.iterrows():
    map_product_dict[int(row['prod_id'])] = row['product_title']
    
    key = int(row['prod_id'])
    vec = list(row[['p'+str(x) for x in list(range(product_tower_dimensions))]])
    
    product_index.add_item(key,vec)

In [14]:
product_index.build(100) # 100 trees
product_index.save('product.tree')


product_index = AnnoyIndex(product_tower_dimensions,  'euclidean')
product_index.load('product.tree')

True

In [15]:
top_k = 20
mat = []
for ix,row in product_embedding.iterrows():
    item = row['product_title']
    mat.append([item] + [map_product_dict[x] for x in product_index.get_nns_by_item(row['prod_id'], top_k+1)[1:]])
    
    if ix == 50:
        break
    
cols = ['product_id']
for i in range(top_k):
    cols += ['nearest_{}'.format(i+1)]

neighbors_product = pd.DataFrame(mat, columns = cols)

display(neighbors_product.head(50))

,product_id,nearest_1,nearest_2,nearest_3,nearest_4,nearest_5,nearest_6,nearest_7,nearest_8,nearest_9,...,nearest_11,nearest_12,nearest_13,nearest_14,nearest_15,nearest_16,nearest_17,nearest_18,nearest_19,nearest_20
0,"Crocs Kids' Handle It Rain Boots , Candy Pink,...","Crocs Kids' Handle It Rain Boots , Candy Pink,...","Crocs Kids' Classic Clog , Black, 9 Toddler","Crocs Kids' Handle It Rain Boots , Navy, 9 Tod...","Crocs Kids' Crocband Clog , Party Pink, 11 Lit...","Crocs Kids' Handle It Rain Boots , Navy, 7 Tod...","Crocs Kids' Crocband Clog , Party Pink, 13 Lit...","Crocs Kids' Classic Clog , Pink Lemonade, 4 To...","Crocs Kids' Unicorn Clog, Ballerina Pink/New M...","Crocs Kids' Handle It Rain Boots , Navy, 12 Li...",...,"Crocs Kids' Classic Clog , Lavender, 7 Toddler","Crocs Kids' Classic Clog , Lavender, 3 Little Kid","Crocs Kids' Classic Clog , Lavender, 4 Toddler","Crocs Kids' Classic Clog , Lavender, 5 Big Kid","Crocs Girls' Isabella Sandal Preschool Flat, p...",Crocs Crocband Sandal Kids Unisex Niños Sandal...,"Crocs Kids' Crocband Sandal, Navy/Red, 10 M US...","Crocs Kids' Classic Glitter Clog , Black, 10 T...","Crocs Unisex Men's and Women's Classic Clog, E...","crocs Bump It Rain Boot (Toddler/Little Kid), ..."
1,"Hatley Kids' Little Classic Rain Boots, Pink &...","Hatley unisex child Classic Boots Raincoat, Na...","Hatley Girls' Toddler Printed Rain Boot, Navy ...",Merrell Kids' Unisex M-Bare Steps H20 Water Sh...,"Burton Mens Reserve Bib Pant, True Black 1, Me...",Bloch Dance Bunnyhop Ballet Slipper (Toddler/L...,"Plano Fusion Bow Case, Pink, Small",Bloch Dance Bunnyhop Ballet Slipper (Toddler/L...,"Bloch Women's Performa Dance Shoe, Theatrical ...",Capezio Daisy 205 Ballet Shoe (Toddler/Little ...,...,Capezio Daisy 205 Ballet Shoe (Toddler/Little ...,Bloch Women's Dansoft Full Sole Leather Ballet...,Pink Motocross Motorcycle Goggles Dirt Bike AT...,Capezio Little Girls' Hold & Stretch Footed Ti...,Capezio Daisy 205 Ballet Shoe (Toddler/Little ...,"Avery Durable View 3 Ring Binder, 1 Inch Slant...",Brewster Home Fashions NuWallpaper Vintage Tin...,"Clarks womens Sillian Paz Slip On Loafer, Blac...","Merrell womens Siren Edge 3 Hiking Shoe, Navy/...",Falari Baseball Cap Adjustable Size for Runnin...
2,Outee Rubber Kids Rain Boots,HDE Womens Rainbow Striped Retro Fashion Dolph...,Utopia Towels 100 Pack Commercial Shop Towels ...,AWAYTR Realistic Printed Butterfly Hair Clip 1...,Dansko Professional Women Mules & Clogs Shoes ...,Expression Tees Womens Keep Calm Plants Have P...,Nike Jumpman Premium 9A1640-681 Laptop Bookbag...,VIPKID Orange Teaching T-Shirt (Unisex) (L),Reebok Men's Shoes Legacy Lifter II Black Oran...,goodr RUNNING SUNGLASSES - (Blue w/Blue Lens),...,"PaddleTek Element Pickleball Paddle, ProPolyCo...",Stance Gamut II Super Invisible Sock Large Whi...,"eclife 24"" Black Bathroom Vanities Set (Contem...",SoB Flowerpot Treeman Baby Groot Succulent Pla...,Abaowedding Lovely Flower Girl Dress Lace Long...,Hanes Men's Cushion Crew Socks (6-Pairs: Shoe ...,TOMORI Anime Cosplay Eye Mask Japanese Animate...,Ourcosplay Green Tsunade Cosplay Costume Coat ...,"HABIT Men's Redbank Hollow Performance Pant, R...",Arus Baby Organic Turkish Cotton Terry Hooded ...
3,Hope & Henry Girls' Red Milano Stitch Cardigan,Hope & Henry Girls' Milano Stitch Cardigan Navy,Hope & Henry Girls' White Cropped Cardigan,Hope & Henry Girls' Navy Pique Tank Dress,Hope & Henry Girls Blue Lightweight Blocked Sk...,Hope & Henry Girls White Bow Shoulder Dress,GRACE KARIN Women's 50s Petticoat Vintage Crin...,Casual Shift Dress for Women with Sleeve Cockt...,ARIAT Women's Ultra Stretch Perfect Rise Katie...,Hanes Women's Mini-Ribbed Cotton Tank_White_XL,...,bebe Girls’ Sandal – Two Strapped Patent Leath...,CAUCASIANS The Original T-Shirt,MeMoi Sheer Footless Capri Shaper Tights Black...,Blue Line American Flag - Support Cops Police ...,My Little Pony Toddler Girls Hooded Costume Sh...,Unique Baby Girls Back to School Bus Shirt Bou...,MBJ W

In [16]:
dataset = pd.merge(data, 
                      product_embedding[['product_id','product_title']].drop_duplicates(), on = ['product_id']
                     )
dataset.head()

,query,product_id,esci_label,split,q0,q1,q2,q3,q4,q5,...,p151,p152,p153,p154,p155,p156,p157,p158,p159,product_title
0,monogram crossbody bags for women,B07SC67L97,E,train,0.007149,-0.036234,-0.021769,0.071798,0.004489,0.046100,...,0.009924,0.015074,0.014845,0.030969,-0.002622,-0.004156,-0.004414,-0.052931,-0.062627,Tommy Hilfiger Jacquard Crossbody Purse With M...
1,monogram crossbody bags for women,B088BMKTVZ,E,train,0.007149,-0.036234,-0.021769,0.071798,0.004489,0.046100,...,0.022348,0.072437,0.017069,0.044085,0.034383,-0.037070,-0.031683,-0.046816,-0.130665,Montana West Leather Cowhide Tote Bag For Wome...
2,monogram crossbody bags for women,B07NVTDGVN,E,train,0.007149,-0.036234,-0.021769,0.071798,0.004489,0.046100,...,-0.007226,0.030359,0.017682,0.054370,-0.040309,-0.028475,0.002260,-0.008909,-0.113362,Tory Burch Women's Gemini Link Canvas Mini Bag...
3,replicia tory burch bags,B07NVTDGVN,E,train,-0.010582,-0.032959,-0.027428,0.031883,-0.000479,0.062785,...,-0.007226,0.030359,0.017682,0.054370,-0.040309,-0.028475,0.002260,-0.008909,-0.113362,Tory Burch Women's Gemini Link Canvas Mini Bag...
4,green advertine crystal,B07H8GJ44Q,E,train,0.025034,-0.025958,-0.021118,0.026299,0.006780,0.022166,...,0.011845,0.031754,0.026041,0.061598,-0.041917,-0.050758,0.015715,-0.050046,-0.078058,"Crystal Glass Diamond Shaped Decoration, Green..."


In [17]:
dataset.esci_label.value_counts()

esci_label
E    79252
I    77845
Name: count, dtype: int64

In [18]:
dataset[dataset.esci_label == 'E'][['query','product_title','split']].sample(20)

,query,product_title,split
104599,nike football indoor shoes,Nike Junior Bomba TF Turf Soccer Shoes (Black/...,train
153860,cotton quilted king beddspread,Laura Ashley Home Bedford Collection Quilt Set...,train
1790,1 fake hawaiian foam flower not clip for ear,Pursuestar 100Pcs White Foam Hawaiian Frangipa...,test
57022,acrylic nail drill,Makartt Nail Drill Electric Nail File Machine ...,train
54409,cat scratch tree,Amazon Basics Extra Large Cat Tree Tower with ...,train
86935,fogless mirror for shower,Fogless Shower Mirror for Shaving with Upgrade...,train
102032,white waterslide paper,Waterslide Paper-20 sheet Inkjet Water Slide P...,test
142677,kangkong seeds for planting,Academyus 500Pcs Water Spinach Seeds Delicious...,train
48837,fender medium guitar picks,Fender 351 Shape Classic Picks (12 Pack) for e...,train
61474,film for polaroid camera,"Polaroid Color Film for 600 Double Pack, 16 Ph...",train


In [19]:
dataset[dataset.esci_label == 'I'][['query','product_title','split']].sample(20)

,query,product_title,split
7910,jeep jk spare tire delete kit,Week 7: Eagles at Redskins Game Highlights,generated
66848,laptop desk,AUSCAMOTEK Hunting Ghillie Suit for Kids Gilly...,generated
70654,do what is right not what is easy sign,"SONGMICS 30-Pack Pants Hangers, 16.7-Inch Long...",generated
128619,werewolf card game,AOWISH 304 Stainless Steel Clevis Slip Hook wi...,generated
154245,hanging circle decorations,Nike Zonal Cooling Dry Half Zip Seamless Golf ...,generated
70472,tub mat without suction cups,"chakme Non-Slip Bathtub Mat, Soft Rubber Bathr...",train
44029,zinus memory foam 12 inch green tea mattress q...,Godox AD400 Pro AD400Pro 400ws GN72 TTL Batter...,generated
137658,lawn speaker,Ben Sherman Oxidized Stainless Steel Geo Bar P...,generated
48626,birds arent real,Programmable Led Scrolling Display PH10 MM Ind...,generated
13597,pool pressure meter,100% Remy Human Hair Ponytail Extension Wrap A...,generated


In [20]:
dataset.split.value_counts()

split
train        69571
generated    67235
test         20291
Name: count, dtype: int64

In [21]:
# Convert 'esci_label' column to binary labels
dataset['binary_label'] = dataset['esci_label'].apply(lambda x: 1 if x == 'E' else 0)

# Split the dataset into training and validation
train_data = dataset[dataset['split'] != 'train']
val_data = dataset[dataset['split'] == 'test']

train_labels = np.array(train_data['binary_label'])
val_labels = np.array(val_data['binary_label'])

train_labels = train_labels.astype('float32')
val_labels = val_labels.astype('float32')



print(train_data.esci_label.value_counts(),
      val_data.esci_label.value_counts())

query_tower_cols = ['q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 
                                  'q9', 'q10', 'q11', 'q12', 'q13', 'q14', 'q15', 'q16', 
                                  'q17', 'q18', 'q19', 'q20', 'q21', 'q22', 'q23', 'q24', 
                                  'q25', 'q26', 'q27', 'q28', 'q29', 'q30', 'q31']

product_tower_cols = ['p0', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10',
                      'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p20',
                      'p21', 'p22', 'p23', 'p24', 'p25', 'p26', 'p27', 'p28', 'p29', 'p30',
                      'p31', 'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p40',
                      'p41', 'p42', 'p43', 'p44', 'p45', 'p46', 'p47', 'p48', 'p49', 'p50',
                      'p51', 'p52', 'p53', 'p54', 'p55', 'p56', 'p57', 'p58', 'p59', 'p60',
                      'p61', 'p62', 'p63', 'p64', 'p65', 'p66', 'p67', 'p68', 'p69', 'p70',
                      'p71', 'p72', 'p73', 'p74', 'p75', 'p76', 'p77', 'p78', 'p79', 'p80',
                      'p81', 'p82', 'p83', 'p84', 'p85', 'p86', 'p87', 'p88', 'p89', 'p90',
                      'p91', 'p92', 'p93', 'p94', 'p95', 'p96', 'p97', 'p98', 'p99', 'p100',
                      'p101', 'p102', 'p103', 'p104', 'p105', 'p106', 'p107', 'p108', 'p109', 'p110',
                      'p111', 'p112', 'p113', 'p114', 'p115', 'p116', 'p117', 'p118', 'p119', 'p120',
                      'p121', 'p122', 'p123', 'p124', 'p125', 'p126', 'p127', 'p128', 'p129', 'p130',
                      'p131', 'p132', 'p133', 'p134', 'p135', 'p136', 'p137', 'p138', 'p139', 'p140',
                      'p141', 'p142', 'p143', 'p144', 'p145', 'p146', 'p147', 'p148', 'p149', 'p150',
                      'p151', 'p152', 'p153', 'p154', 'p155', 'p156', 'p157', 'p158', 'p159'
                     ]



# Prepare input data for training and validation
train_inputs = [
    np.array(train_data[query_tower_cols]),
    
    np.array(train_data[product_tower_cols])
    
]

val_inputs = [
    np.array(val_data[query_tower_cols]),
    
    np.array(val_data[product_tower_cols])
]

esci_label
I    69956
E    17570
Name: count, dtype: int64 esci_label
E    17570
I     2721
Name: count, dtype: int64


# Two-tower model architecture for Product Search we implemented as per the availability of the data.

![Alt Text](two-tower-model-architecture-implemented.png)

In [22]:
import tensorflow as tf
from keras.layers import Input, Dense, Dot, Lambda
from keras.models import Model


# Define Input Layers
input_query = Input(shape=(query_tower_dimensions,), name='input_query')
input_product = Input(shape=(product_tower_dimensions,), name='input_product')

# Apply a Dense Layer to Reduce Embedding Dimensionality
embedding_dim = 16
final_query_embedding = Dense(embedding_dim, activation='linear', name='embedding_layer_query')(input_query)
final_product_embedding = Dense(embedding_dim, activation='linear', name='embedding_layer_product')(input_product)

# Normalize Embeddings (L2 Normalization - cossine similarity - ensures all embeddings have a unit length (magnitude = 1)) 
normalized_query = Lambda(lambda x: tf.keras.backend.l2_normalize(x, axis=-1), name='normalize_query')(final_query_embedding)
normalized_product = Lambda(lambda x: tf.keras.backend.l2_normalize(x, axis=-1), name='normalize_product')(final_product_embedding)

# Compute Cosine Similarity
# This step is the core of the retrieval process, as it measures how similar a query is to a product.
cosine_similarity = Dot(axes=1, normalize=True, name='cosine_similarity')([normalized_product, normalized_query])

# Build the model
model = Model(inputs=[input_query, input_product], outputs=cosine_similarity)

# Summary of the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_product       │ (None, 160)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_query         │ (None, 32)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_layer_pr… │ (None, 16)        │      2,576 │ input_product[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_layer_qu… │ (None, 16)        │        528 │ input_query[0][0] │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalize_product   │ (None, 16)        │          0 │ embedding_layer_… │
│ (Lambda)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalize_query     │ (None, 16)        │          0 │ embedding_layer_… │
│ (Lambda)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cosine_similarity   │ (None, 1)         │          0 │ normalize_produc… │
│ (Dot)               │                   │            │ normalize_query[… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,104 (12.12 KB)

 Trainable params: 3,104 (12.12 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [24]:
# Define a model checkpoint callback to save the best model weights during training
checkpoint_path = 'best_model_weights.h5'  # Specify the path to save the best model
model_checkpoint = ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss',  # Monitor validation loss
    verbose=1,
    save_best_only=True,  # Save only the best model
    mode='min'  # Minimize the validation loss
)

# Compile the model with 'binary_crossentropy' loss as a string
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


# Fit the model with model checkpoint callback
history = model.fit(
    # Training data and parameters
    train_inputs, train_labels, epochs=100, batch_size=64, 
    validation_data=(val_inputs, val_labels),
    callbacks=[model_checkpoint],
#     class_weight=dict(enumerate(class_weights))
    # Giving positive class more importance to positive labels to ensure precision is high.
    class_weight=dict(enumerate([1,2]))
                   )

Epoch 1/100
1355/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 820us/step - accuracy: 0.7643 - loss: 0.7895
Epoch 1: val_loss improved from inf to 0.88359, saving model to best_model_weights.h5


1368/1368 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7645 - loss: 0.7890 - val_accuracy: 0.3351 - val_loss: 0.8836
Epoch 2/100
1347/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 823us/step - accuracy: 0.7890 - loss: 0.6526
Epoch 2: val_loss improved from 0.88359 to 0.87171, saving model to best_model_weights.h5


1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7889 - loss: 0.6526 - val_accuracy: 0.4363 - val_loss: 0.8717
Epoch 3/100
1357/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 779us/step - accuracy: 0.7875 - loss: 0.6385
Epoch 3: val_loss improved from 0.87171 to 0.80233, saving model to best_model_weights.h5


1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 966us/step - accuracy: 0.7874 - loss: 0.6386 - val_accuracy: 0.4708 - val_loss: 0.8023
Epoch 4/100
1358/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 780us/step - accuracy: 0.7930 - loss: 0.6240
Epoch 4: val_loss did not improve from 0.80233
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 952us/step - accuracy: 0.7930 - loss: 0.6240 - val_accuracy: 0.5754 - val_loss: 0.8112
Epoch 5/100
1353/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 858us/step - accuracy: 0.7676 - loss: 0.6431
Epoch 5: val_loss did not improve from 0.80233
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7674 - loss: 0.6435 - val_accuracy: 0.3536 - val_loss: 0.8742
Epoch 6/100
1316/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step - accuracy: 0.7515 - loss: 0.6728
Epoch 6: val_loss improved from 0.80233 to 0.75945, saving model to best_model_weights.h5


1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 998us/step - accuracy: 0.7528 - loss: 0.6714 - val_accuracy: 0.5949 - val_loss: 0.7595
Epoch 7/100
1359/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 815us/step - accuracy: 0.7954 - loss: 0.5953
Epoch 7: val_loss did not improve from 0.75945
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7954 - loss: 0.5954 - val_accuracy: 0.6012 - val_loss: 0.7715
Epoch 8/100
1304/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step - accuracy: 0.8023 - loss: 0.5928
Epoch 8: val_loss did not improve from 0.75945
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 982us/step - accuracy: 0.8016 - loss: 0.5942 - val_accuracy: 0.3542 - val_loss: 0.8523
Epoch 9/100
1354/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 783us/step - accuracy: 0.8007 - loss: 0.6403
Epoch 9: val_loss did not improve from 0.75945
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 947us/step - accuracy: 0.8008 - loss: 0.6401 - val_accuracy: 0.5320 - val_loss: 0.7861
Epoch 10/100
1343/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - accuracy: 0.8165 - loss: 0.5783
Epoch 10: va

...

1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 921us/step - accuracy: 0.8122 - loss: 0.5762 - val_accuracy: 0.6889 - val_loss: 0.6683
Epoch 15/100
1362/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 815us/step - accuracy: 0.8029 - loss: 0.5749
Epoch 15: val_loss did not improve from 0.66833
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 984us/step - accuracy: 0.8029 - loss: 0.5749 - val_accuracy: 0.6878 - val_loss: 0.6728
Epoch 16/100
1323/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 841us/step - accuracy: 0.8102 - loss: 0.5541
Epoch 16: val_loss did not improve from 0.66833
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8104 - loss: 0.5543 - val_accuracy: 0.5768 - val_loss: 0.8081
Epoch 17/100
1309/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 770us/step - accuracy: 0.7880 - loss: 0.6370
Epoch 17: val_loss did not improve from 0.66833
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 949us/step - accuracy: 0.7885 - loss: 0.6365 - val_accuracy: 0.5027 - val_loss: 0.7905
Epoch 18/100
1330/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 833us/step - accuracy: 0.8214 - loss: 0.5708
Epoch 

1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 949us/step - accuracy: 0.8092 - loss: 0.5676 - val_accuracy: 0.7057 - val_loss: 0.6336
Epoch 22/100
1319/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 765us/step - accuracy: 0.8209 - loss: 0.5479
Epoch 22: val_loss did not improve from 0.63360
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 933us/step - accuracy: 0.8206 - loss: 0.5484 - val_accuracy: 0.5530 - val_loss: 0.7543
Epoch 23/100
1350/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 783us/step - accuracy: 0.7998 - loss: 0.5898
Epoch 23: val_loss did not improve from 0.63360
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 946us/step - accuracy: 0.7998 - loss: 0.5896 - val_accuracy: 0.6541 - val_loss: 0.7039
Epoch 24/100
1306/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 771us/step - accuracy: 0.8288 - loss: 0.5264
Epoch 24: val_loss did not improve from 0.63360
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 970us/step - accuracy: 0.8288 - loss: 0.5268 - val_accuracy: 0.6735 - val_loss: 0.7542
Epoch 25/100
1360/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 860us/step - accuracy: 0.8112 - loss: 0.5535
Epoc

1368/1368 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8266 - loss: 0.5370 - val_accuracy: 0.7020 - val_loss: 0.6253
Epoch 34/100
1353/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8339 - loss: 0.5242
Epoch 34: val_loss did not improve from 0.62527
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8339 - loss: 0.5243 - val_accuracy: 0.5263 - val_loss: 0.7806
Epoch 35/100
1355/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8252 - loss: 0.5609
Epoch 35: val_loss did not improve from 0.62527
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8251 - loss: 0.5610 - val_accuracy: 0.4907 - val_loss: 0.8097
Epoch 36/100
1334/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7916 - loss: 0.6079
Epoch 36: val_loss did not improve from 0.62527
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7919 - loss: 0.6074 - val_accuracy: 0.5919 - val_loss: 0.7207
Epoch 37/100
1343/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8342 - loss: 0.5321
Epoch 37: val_loss d

1368/1368 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8536 - loss: 0.4865 - val_accuracy: 0.7168 - val_loss: 0.6229
Epoch 63/100
1329/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 990us/step - accuracy: 0.8518 - loss: 0.4792
Epoch 63: val_loss did not improve from 0.62290
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8519 - loss: 0.4792 - val_accuracy: 0.6846 - val_loss: 0.6791
Epoch 64/100
1349/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8019 - loss: 0.6154
Epoch 64: val_loss did not improve from 0.62290
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8017 - loss: 0.6157 - val_accuracy: 0.5838 - val_loss: 0.7707
Epoch 65/100
1312/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 769us/step - accuracy: 0.7803 - loss: 0.6292
Epoch 65: val_loss did not improve from 0.62290
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 932us/step - accuracy: 0.7799 - loss: 0.6291 - val_accuracy: 0.6655 - val_loss: 0.6565
Epoch 66/100
1363/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 741us/step - accuracy: 0.7841 - loss: 0.6085
Epoch 66: va

1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 927us/step - accuracy: 0.8247 - loss: 0.5603 - val_accuracy: 0.7211 - val_loss: 0.5942
Epoch 69/100
1360/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7241 - loss: 0.6539
Epoch 69: val_loss did not improve from 0.59425
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7243 - loss: 0.6539 - val_accuracy: 0.6285 - val_loss: 0.6882
Epoch 70/100
1319/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7797 - loss: 0.6280
Epoch 70: val_loss did not improve from 0.59425
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7801 - loss: 0.6278 - val_accuracy: 0.5565 - val_loss: 0.7628
Epoch 71/100
1329/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step - accuracy: 0.8102 - loss: 0.6073
Epoch 71: val_loss did not improve from 0.59425
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 941us/step - accuracy: 0.8103 - loss: 0.6071 - val_accuracy: 0.5537 - val_loss: 0.7743
Epoch 72/100
1336/1368 ━━━━━━━━━━━━━━━━━━━━ 0s 791us/step - accuracy: 0.8292 - loss: 0.5716
Epoch 72: va

In [25]:
model.load_weights(checkpoint_path)

In [26]:
product_model = Model(inputs=[input_query, input_product],
                         outputs=model.get_layer('normalize_product').output
                   )

In [27]:
input_data_product = [
    np.array([np.array([0] * query_tower_dimensions)] * product_embedding.shape[0]),
    
    np.array(product_embedding[product_tower_cols])
    
]

product_embeddings = product_model.predict(input_data_product)
len(product_embeddings)

4615/4615 ━━━━━━━━━━━━━━━━━━━━ 3s 701us/step


147673

In [28]:
product_embeddings_model = pd.DataFrame(product_embeddings, columns = [f'p{x}'for x in range(embedding_dim)] )

product_embeddings_model['product_id'] = product_embedding['product_id']
product_embeddings_model['product_title'] = product_embedding['product_title']
product_embeddings_model['prod_id'] = product_embedding['prod_id']

product_embeddings_model.head()

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,product_id,product_title,prod_id
0,-0.037311,-0.134417,0.063639,0.409084,0.366814,-0.186275,0.070886,-0.105568,0.006181,-0.440595,0.205893,0.384778,0.141929,0.329174,0.206561,-0.260663,B01A5DU2TA,"Crocs Kids' Handle It Rain Boots , Candy Pink,...",0
1,-0.046316,-0.179502,0.103145,0.288750,0.329827,-0.342277,0.051158,-0.102361,0.085330,-0.284540,0.254343,0.244518,0.129858,0.453425,0.333281,-0.293814,B07346RPKN,"Hatley Kids' Little Classic Rain Boots, Pink &...",1
2,-0.087014,-0.124317,0.134761,0.343562,0.336178,-0.292361,-0.000226,-0.073460,0.063324,-0.312179,0.322499,0.343832,0.266358,0.323102,0.197201,-0.314596,B078TCVZ4V,Outee Rubber Kids Rain Boots,2
3,-0.067304,-0.053431,0.258928,-0.098213,0.374820,-0.105259,-0.294801,-0.140164,0.150150,-0.411557,0.031838,0.424915,-0.009153,0.312527,0.119049,-0.415190,B07BR3PPDR,Hope & Henry Girls' Red Milano Stitch Cardigan,3
4,-0.048876,-0.230848,0.027356,0.035114,0.402428,-0.163950,-0.147398,-0.362078,0.151802,-0.268476,0.226277,0.242675,0.041441,0.477133,0.362939,-0.185388,B07H5C4P19,Spring&Gege Youth Solid Full Zipper Hoodies So...,4


In [29]:
pm = AnnoyIndex(embedding_dim, 'dot')
mp_product_dict = {}

for ix,row in product_embeddings_model.iterrows():
    mp_product_dict[int(row['prod_id'])] = row['product_title']
    
    key = int(row['prod_id'])
    vec = list(row[['p'+str(x) for x in list(range(embedding_dim))]])
    
#     print(key,vec)
    pm.add_item(key,vec)

In [30]:
pm.build(100) # 100 trees
pm.save('product_model.tree')

# pm = AnnoyIndex(embedding_dim,  'euclidean')
pm = AnnoyIndex(embedding_dim,  'dot')
pm.load('product_model.tree')

True

In [31]:
top_k = 20
mat = []
for ix,row in product_embeddings_model.iterrows():
    item = row['product_title']
    mat.append([item] + [mp_product_dict[x] for x in pm.get_nns_by_item(row['prod_id'], top_k+1)[1:]])
    
    if ix == 50:
        break
    
cols = ['product_id']
for i in range(top_k):
    cols += ['nearest_{}'.format(i+1)]

print(cols)

df_neighbors3 = pd.DataFrame(mat, columns = cols)

display(df_neighbors3.head(50))

['product_id', 'nearest_1', 'nearest_2', 'nearest_3', 'nearest_4', 'nearest_5', 'nearest_6', 'nearest_7', 'nearest_8', 'nearest_9', 'nearest_10', 'nearest_11', 'nearest_12', 'nearest_13', 'nearest_14', 'nearest_15', 'nearest_16', 'nearest_17', 'nearest_18', 'nearest_19', 'nearest_20']


,product_id,nearest_1,nearest_2,nearest_3,nearest_4,nearest_5,nearest_6,nearest_7,nearest_8,nearest_9,...,nearest_11,nearest_12,nearest_13,nearest_14,nearest_15,nearest_16,nearest_17,nearest_18,nearest_19,nearest_20
0,"Crocs Kids' Handle It Rain Boots , Candy Pink,...","Crocs Kids' Handle It Rain Boots , Candy Pink,...","Crocs Kids' Classic Clog , Lavender, 3 Little Kid","Crocs Kids' Handle It Rain Boots , Navy, 9 Tod...","Crocs Kids' Handle It Rain Boots , Navy, 7 Tod...","Crocs Kids' Handle It Rain Boots , Cerulean Bl...","Crocs Kids' Handle It Rain Boots , Cerulean Bl...","Crocs Kids' Classic Clog , Lavender, 7 Toddler","Crocs Kids' Crocband Clog , Ice Blue/White, 11...",Miraculous Ladybug Boys' Cat Noir Pajamas Size...,...,"Crocs Kids' Classic Clog , Lavender, 4 Toddler",Crocs Kids' Classic Realtree Clog | Camo Shoes...,Africa Toto Life Soundtrack funny sarcastic shirt,"The Children's Place boys Pull On Cargo Pants,...","Crocs Kids' Classic Clog , Slate Grey, 3 Littl...",The Children's Place Baby Boys' Shark Two Piec...,Misfits Splatter Pullover Hoodie,"Crocs Kids' Classic Glitter Clog , Black, 10 T...","Crocs Kids' Classic Clog , Black, 9 Toddler","Crocs Kids' Classic Clog , Pink Lemonade, 4 To..."
1,"Hatley Kids' Little Classic Rain Boots, Pink &...",Skechers Men Relaxed Fit¿: Relment - Pelmo Chu...,Unlisted by Kenneth Cole Mens UN-Anchor Fabric...,You Cant Spell Sausage Without USA Funny 4th O...,"Crocs Kids' Handle It Rain Boots , Navy, 9 Tod...","Crocs Kids' Handle It Rain Boots , Navy, 7 Tod...","Crocs Kids' LiteRide Clog , Navy/White, 3 Litt...",Charles River Apparel unisex child Pacific Pon...,"Joules Girls JNR Welly Print Rain Boot, Navy M...","Crocs Kids' Handle It Rain Boots , Navy, 13 Li...",...,"Splash About Go Splash Swim Vest, Vintage Moby...",Safavieh Blossom Collection BLM402A Handmade P...,Eddie Bauer Home | Fairview Collection | 100% ...,Children Toddlers Infant & Baby Mittens Waterp...,Carhartt unisex baby Signature Canvas Baseball...,HIRIRI Baby Toddlers Cable Knit Knee High Sock...,"Crocs Kids' Handle It Rain Boots , Navy, 12 Li...",Western Chief Kids Neoprene Kid's Boots (Toddl...,TeesNow Vintage Surely Not Everyone was Kung F...,New Balance Kid's Fresh Foam Roav V1 Lace-Up S...
2,Outee Rubber Kids Rain Boots,'47 New York Yankees Brand Metallic Clean Up S...,jotkdje Bad Bunny Cap Embroidered Baseball Dad...,NFL Siskiyou Sports Fan Shop Dallas Cowboys Be...,New Balance Kid's Fresh Foam Roav V1 Lace-Up S...,"Crocs Kids' Handle It Rain Boots , Navy, 9 Tod...",I'm not a cat shirt I am not a cat shirt funny...,"I'M HERE LIVE, I'M NOT A CAT TEXAS LAWYERS KIT...",Native Kids Shoes Charley (Toddler/Little Kid)...,Dearfoams Boys' Kid's Camo and Fleece Clog Sli...,...,Kereda Little Kids Girls Slouch Crew Socks Tod...,"Crocs Kids' Handle It Rain Boots , Navy, 7 Tod...","Crocs Kids' Handle It Rain Boots , Cerulean Bl...",The Children's Place Baby Boys' Shark Two Piec...,NFL Siskiyou Sports Womens New York Giants Dan...,Judy Blume Boxed Set: Fudge-a-Mania; Otherwise...,"Native Kids unisex-baby Jefferson Water Shoe,p...",New Era 59Fifty Hat MLB Basic Boston Red Sox B...,Snort Pines Not Lines Funny Scout Panda T-Shirt,Weestep Toddler Little Kid Boys and Girls Slip...
3,Hope & Henry Girls' Red Milano Stitch Cardigan,"Tory Burch Women's Minnie Travel Ballet Flats,...",Hurley Men's Premium Icon Slash Graphic Short ...,Hope & Henry Girls' Milano Stitch Cardigan Navy,Eliza J Women's Long Sleeve Polka Dot Maxi Dre...,GRACE KARIN Women Sexy Wrap Bodycon Dress V Ne...,adidas RapidaRun Laceless Knit Juniors Sneaker...,"Calvin Klein Baby Girls Dress, Winter White/Me...",c Men's Hoodies Pullover Workout Gym Sport Swe...,Womens University of South Florida USF Bulls N...,...,Elite Fan Shop Penn State Nittany Lions Long S...,"adidas Boys' Big Active Athletic Shorts, PRO S...",Calvin Klein Women's Carousel Triangle Bralett...,"Calvin Klein Men's Steel Micro Boxer Briefs, d...","Columbia Girls' Little Arcadia Jacket, Dark Li...","

In [32]:
query_model = Model(inputs=[input_query, input_product],
                         outputs=model.get_layer('normalize_query').output
                   )

In [33]:
input_data_query = [
    
    np.array(query[query_tower_cols]),
    np.array([np.array([0] * product_tower_dimensions)] * query.shape[0]),
    
]


query_embeddings = query_model.predict(input_data_query)
len(query_embeddings)

2212/2212 ━━━━━━━━━━━━━━━━━━━━ 1s 592us/step


70756

In [34]:
df_query_embeddings_model = pd.DataFrame(query_embeddings, columns = [f'q{x}'for x in range(embedding_dim)] )

df_query_embeddings_model['query'] = query['query']
df_query_embeddings_model['qid'] = query['que_id']

df_query_embeddings_model.head()

,q0,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12,q13,q14,q15,query,qid
0,0.198352,-0.219821,0.276097,0.212317,0.563120,-0.416996,-0.134090,-0.343299,-0.033946,0.024546,-0.023513,0.089179,-0.179250,0.266443,-0.224783,-0.017096,child proof cabinet locks,0
1,-0.195260,-0.012324,0.173923,-0.165633,0.210703,-0.061539,-0.269781,-0.313340,-0.062814,-0.161074,0.025255,0.479067,-0.328769,0.233869,0.230660,-0.457023,ankle stockings for women sheer,1
2,-0.247323,0.146933,0.529848,0.165305,0.470479,0.136130,0.189461,0.048759,-0.374416,-0.261403,0.130865,-0.265010,-0.167572,0.072205,0.042996,0.002370,gluten free snacks,2
3,-0.055286,0.026321,0.492001,-0.084946,0.436889,0.059578,-0.205115,-0.021431,-0.144545,-0.463972,0.205088,0.139664,-0.373771,0.100752,0.132697,-0.211714,hair geow,3
4,0.104719,-0.333596,0.282115,-0.185854,0.406185,-0.258851,-0.150777,-0.596709,-0.122781,-0.248925,-0.095930,0.032784,0.056322,-0.154181,-0.091366,-0.173849,"1 by one, amplified, outdoor hdtv antenna",4


In [35]:
qm = AnnoyIndex(embedding_dim, 'dot')
mp_query_dict = {}

for ix,row in df_query_embeddings_model.iterrows():
    mp_query_dict[row['qid']] = row['query']
    
    key = int(row['qid'])
    vec = list(row[['q'+str(x) for x in list(range(embedding_dim))]])
    
#     print(key,vec)
    qm.add_item(key,vec)

In [36]:
qm.build(100) # 100 trees
qm.save('query_model.tree')

# qm = AnnoyIndex(embedding_dim,  'euclidean')
qm = AnnoyIndex(embedding_dim,  'dot')
qm.load('query_model.tree')

True

In [37]:
top_k = 20
mat = []
for ix,row in df_query_embeddings_model.iterrows():
    item = row['query']
    mat.append([item] + [mp_query_dict[x] for x in qm.get_nns_by_item(row['qid'], top_k+1)[1:]])
    
    if ix == 50:
        break
    
cols = ['query_id']
for i in range(top_k):
    cols += ['nearest_{}'.format(i+1)]

print(cols)

df_neighbors4 = pd.DataFrame(mat, columns = cols)

display(df_neighbors4.head(50))

['query_id', 'nearest_1', 'nearest_2', 'nearest_3', 'nearest_4', 'nearest_5', 'nearest_6', 'nearest_7', 'nearest_8', 'nearest_9', 'nearest_10', 'nearest_11', 'nearest_12', 'nearest_13', 'nearest_14', 'nearest_15', 'nearest_16', 'nearest_17', 'nearest_18', 'nearest_19', 'nearest_20']


,query_id,nearest_1,nearest_2,nearest_3,nearest_4,nearest_5,nearest_6,nearest_7,nearest_8,nearest_9,...,nearest_11,nearest_12,nearest_13,nearest_14,nearest_15,nearest_16,nearest_17,nearest_18,nearest_19,nearest_20
0,child proof cabinet locks,child locks for cabinets,cabinet locks,door child lock,kids safety locks for cabinets,cabinet handles,baby safety locks for cabinets,magnetic child safety cabinet locks,travel door lock,door slam stopper,...,baby proofing cabinet lock bows,infant girl toys,digger toys for boys,crafts for kids,master lock door,safety door latch for kids,door lock,treasure box toys for classroom,vintage signs for kitchen,electromagnetic door lock
1,ankle stockings for women sheer,spider jackets for men,light jackets for women,brown high boots for women,white fabric belts for women,cargo pants for men,womens belt for dress,cargo pants for women,short boots for women,yellow boots for women thigh high,...,jogger pants for men,linen pants women,harem pants for women,back slit shirts for women,off shoulder sweatshirt for women,womens learher skirt brown,mid calf boots for women,fleece jacket women,flared pants for women,waist apron with pockets
2,gluten free snacks,jerky without sugar,miltons gluten free crackers,gluten free lemon cookie,papaya enzymes chewable,wrinkle reducing cream,gluten free sugar free candy,gluten free dumpling wrappers,boiled peanuts,knorr rice sides gluten free,...,vegetable peeler,vegetable flakes for soup,jewlery cleaner precious,cucumber face wash,sugar in the raw,sprinkles are for cupcakes not toilet seats,sugar substitute,mederma quick dry oil,bunn coffee filters,salt and pepper shakers with holder
3,hair geow,japonesque sponges,didlo large thick huge,maraschino cherries,contigo sheffield water bottle,mens pork pie hat stingy brim,parakeet silo feeder clean,smartbones large,cz dangle earrings,puma shoes men,...,snow blower ariens,nose strips for blackheads,inuyasha sweater,queenie bags,joker zippo,ghee bulk,frank damazio,scungilli canned,sour belts,spoozie booster pads
4,"1 by one, amplified, outdoor hdtv antenna",smart meal prep for beginners,makeup organizer travel,happy ever after,copic ciao art markers,evulution bcaa,show your mood without saying a word,led name badge,my life would suck without you,away the carry on,...,entertaining evil,motivational calendar,avent bottles,chtistmas decoration lights,11. he y treverse matts,adorime,amazing energy,bra,within temptation,echo buds accesories
5,adidas sweats slim,babybliss pro gold fx,katy perry,adidas original superstar women,adidas pants boys,billie eilish hoodie,adidas running shoes mens,fila sweatpants,hot diamond process sexy dress party club nigh...,toby keith greatest hits cd,...,metallica baby clothes,adidas tiro 17,levis 721 high rise skinny,rainbird 10 psi,barbie chelsey,adidas soccer cleats,adidas pants,tupac shirt,back to december taylor swift,adidas shoes pw hu
6,beta sitosterol now,obagi clenziderm acne,gelato on my mind opi,ropa erótica,dmall phollips screw driver,probiotoc,kiss i gloo,neil medi sinus rinse packets,lansinoh therapearl 3-in-1 hot or cold breast ...,sanerdirect diamond painting,...,the new great depression james rickards,probiotic multienzyme,working with winston,lube for sex,lip liner,strivectin,infuser,emmi-dent,bic gelocity,dual case for lg stylo 5
7,iphone 11 pro,screen protector iphone x,apple ipad 9.7 case,screen protector iphone 6s,red case iphone 7,case for iphone 7,defender series iphone xr,phone screen protector iphone 7,phone cases iphone 7,phone protector iphone xr,...,phone case iphone x,phone case iphone 8,battery case for iphone 6,crescent moon iphone 7 plus case,case for iphone 6,phone case iphone 6,privacy screen iphone xr,otterbox case iphone 8,apple case for iphone x,iphone xs renewed
8,large training pads for dogs,hebrew nationalhot dogs,dog training videos,roping dummy,dog beds for large dogs,lift blocks,5/8 dock lines for boats,12 ft trampoline with enclosure,exercise mats,dog costum

In [38]:
product_search_queries = [
    "Red dress for a wedding",
    "Best laptop under $1000",
    "Nike running shoes for women",
    "iPhone 13 reviews",
    "Men's winter jackets on sale",
    "Top-rated kitchen appliances",
    "Summer dresses for women",
    "Samsung TV specifications",
    "Gift ideas for anniversary",
    "XYZ brand headphones",
    "Women's shoes size 8",
    "Deals on smartphones",
    "Men's formal suits",
    "Fitness trackers with heart rate monitor",
    "Digital cameras with 4K video",
    "Wireless gaming mouse",
    "Outdoor camping gear",
    "Bluetooth earbuds under $50",
    "Designer handbags on clearance",
    "Gaming laptops with RTX graphics",
    "Organic skincare products",
    "Smartphones with long battery life",
    "Running shoes for flat feet",
    "Laptop deals for students",
    "Wireless noise-canceling headphones",
    "Digital cameras for beginners",
    "Best coffee makers under $50",
    "Latest fashion trends for summer",
    "Top-rated gaming keyboards",
    "Dining room furniture sets",
    "Winter coats for kids",
    "Men's watches with leather straps",
    "Home gym equipment for small spaces",
    "Women's handbags on sale",
    "Bluetooth speakers with waterproof features",
    "Outdoor patio furniture",
    "Affordable fitness trackers",
    "Cookware sets for induction cooktops",
    "Portable chargers for smartphones",
    "Desktop computers for gaming",
]
product_search_queries

['Red dress for a wedding',
 'Best laptop under $1000',
 'Nike running shoes for women',
 'iPhone 13 reviews',
 "Men's winter jackets on sale",
 'Top-rated kitchen appliances',
 'Summer dresses for women',
 'Samsung TV specifications',
 'Gift ideas for anniversary',
 'XYZ brand headphones',
 "Women's shoes size 8",
 'Deals on smartphones',
 "Men's formal suits",
 'Fitness trackers with heart rate monitor',
 'Digital cameras with 4K video',
 'Wireless gaming mouse',
 'Outdoor camping gear',
 'Bluetooth earbuds under $50',
 'Designer handbags on clearance',
 'Gaming laptops with RTX graphics',
 'Organic skincare products',
 'Smartphones with long battery life',
 'Running shoes for flat feet',
 'Laptop deals for students',
 'Wireless noise-canceling headphones',
 'Digital cameras for beginners',
 'Best coffee makers under $50',
 'Latest fashion trends for summer',
 'Top-rated gaming keyboards',
 'Dining room furniture sets',
 'Winter coats for kids',
 "Men's watches with leather straps",


In [44]:
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained model (you can choose from various models like BERT, RoBERTa, etc.)
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
model_sentence_transformers = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [45]:
import numpy as np

def reshape_array(input_array, d):
    k, _ = input_array.shape
    new_array = np.zeros((k, d))
    
    for i in range(k):
        for j in range(d):
            start_idx = j * (768 // d)
            end_idx = (j + 1) * (768 // d) if j < (d - 1) else 768
            chunk = input_array[i, start_idx:end_idx]
            new_array[i, j] = np.mean(chunk)
    
    return new_array

In [46]:
query_max_words = 10
product_title_max_words = 50
product_description_max_words = 20
product_bullet_point_max_words = 50
product_brand_max_words = 5
product_color_max_words = 5

In [47]:
query_dim = 32

cols = ['q' + str(x) for x in list(range(0, query_dim))] + ['query']

sentences = product_search_queries

sentence_embeddings = model_sentence_transformers.encode(sentences, convert_to_tensor=True)

sentence_embeddings = reshape_array(sentence_embeddings.cpu().numpy(), query_dim)
    
df_query_test_actual = pd.DataFrame(sentence_embeddings)

df_query_test_actual.columns = query_tower_cols
df_query_test_actual['query'] = product_search_queries

df_query_test_actual.head(10)

/Users/mariananeto/Documents/GitHub/two-tower-retrieval-system/.venv/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/mariananeto/Documents/GitHub/two-tower-retrieval-system/.venv/lib/python3.11/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


,q0,q1,q2,q3,q4,q5,q6,q7,q8,q9,...,q23,q24,q25,q26,q27,q28,q29,q30,q31,query
0,-0.004781,0.002380,-0.002925,-0.009321,0.001733,-0.010312,0.008887,0.016795,0.015969,0.008287,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red dress for a wedding
1,0.007920,0.007424,-0.000631,-0.008106,0.008224,-0.009331,-0.008879,0.000196,0.007047,0.013668,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Best laptop under $1000
2,-0.004958,-0.007405,0.002094,0.009622,0.011420,-0.007282,0.007531,0.000195,0.007078,0.004819,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nike running shoes for women
3,0.002123,-0.003557,-0.000962,0.008468,0.017000,-0.010397,-0.005630,0.001745,-0.017004,-0.003496,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,iPhone 13 reviews
4,-0.002479,-0.001443,0.006723,-0.010479,-0.008341,0.012055,-0.000161,0.011393,0.020483,0.008022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Men's winter jackets on sale
5,-0.007662,-0.008722,-0.011714,-0.002041,-0.004252,0.017269,-0.009400,0.013196,-0.002697,0.007032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Top-rated kitchen appliances
6,0.008600,-0.005502,-0.005778,-0.002773,0.002431,-0.003092,0.014936,0.001300,0.018078,0.000258,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer dresses for women
7,-0.013254,-0.003586,-0.002379,0.001163,0.004578,-0.003027,0.003249,0.013432,-0.012253,0.006608,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Samsung TV specifications
8,-0.003872,0.012509,-0.000483,-0.016884,0.012719,0.009545,0.009701,-0.005805,0.007914,-0.012715,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gift ideas for anniversary
9,0.001816,-0.008768,0.002865,-0.009514,-0.001542,-0.004419,-0.009681,-0.001781,0.012307,0.008703,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XYZ brand headphones


In [48]:
input_data_query = [
    
    np.array(df_query_test_actual[query_tower_cols]),
    np.array([np.array([0] * product_tower_dimensions)] * df_query_test_actual.shape[0]),
    
]


query_embeddings = query_model.predict(input_data_query)
len(query_embeddings)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step 


40

In [49]:
cols = [f'q{x}'for x in range(embedding_dim)]

df_query_test = pd.DataFrame(query_embeddings, columns = cols)
df_query_test['query'] = product_search_queries

df_query_test.head(50)

,q0,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12,q13,q14,q15,query
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red dress for a wedding
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Best laptop under $1000
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nike running shoes for women
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,iPhone 13 reviews
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Men's winter jackets on sale
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Top-rated kitchen appliances
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer dresses for women
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Samsung TV specifications
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gift ideas for anniversary
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XYZ brand headphones


In [50]:
results = []
top_k = 20

desc = []
for i in range(top_k):
    desc.append(f'product_title_{(i+1)}')
desc = ['query'] + desc

for ix,row in df_query_test.iterrows():
    qvec = [f'q{x}' for x in list(range(embedding_dim))]
    qvec = row[qvec]
    similar_vector_ids,similar_distances = pm.get_nns_by_vector(qvec, top_k, include_distances=True)

#     print(similar_vector_ids, similar_distances)

    similar_vector_product_title = [mp_product_dict[x] for x in similar_vector_ids]
    results.append([row['query']] + similar_vector_product_title)

df_results = pd.DataFrame(results, columns =  desc)

df_results.head(50)

/var/folders/pw/sbpycr1d17b1j4255gwjl17c0000gn/T/ipykernel_93064/1790015180.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  similar_vector_ids,similar_distances = pm.get_nns_by_vector(qvec, top_k, include_distances=True)
/var/folders/pw/sbpycr1d17b1j4255gwjl17c0000gn/T/ipykernel_93064/1790015180.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  similar_vector_ids,similar_distances = pm.get_nns_by_vector(qvec, top_k, include_distances=True)
/var/folders/pw/sbpycr1d17b1j4255gwjl17c0000gn/T/ipykernel_93064/1790015180.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future v

,query,product_title_1,product_title_2,product_title_3,product_title_4,product_title_5,product_title_6,product_title_7,product_title_8,product_title_9,...,product_title_11,product_title_12,product_title_13,product_title_14,product_title_15,product_title_16,product_title_17,product_title_18,product_title_19,product_title_20
0,Red dress for a wedding,"Bottle Cutter & Glass Cutter Kit, for Cutting ...",InSinkErator PRO750 with Cord Pro Series 3/4 H...,"LETMY LED Tactical Flashlight, Ultra Bright 20...","OEMTOOLS 24221 Torque Limiting Extension Set, ...",amFilm Protector Pantalla (2 Piezas) para Nint...,amFilm Tempered Glass Screen Protector for Nin...,Yodoit for iPhone 6s Plus Screen Replacement T...,バックトゥザフューチャー グッズ ホバーボード おもちゃ 玩具 ＳＦ 映画 ＢＴＴＦ ＢＴＦ...,Officially Licensed Back to the Future 1:1 Sca...,...,"Gruper Yoga Mat Non Slip, Eco Friendly Fitness...",2枚入り「amFilm 」Nintendo Switch 用 ガラスフィルム 強化ガラス 保...,Yodoit iPhone6s Plus フロントパネル 画面 液晶 取り付け簡単化 (ホー...,"TECCPO Cordless Drill, Drill Driver Set with 2...",【Caseology】 AirPods Pro ケース カバー 耐久性 AirPodsPro...,Caseology Vault for Apple Airpods Pro Case (20...,Miniafinador de pinza PW-CT-12 NS de Planet Waves,D'Addario Guitar Tuner - Micro Headstock Tuner...,D'Addario ダダリオ ヘッドストックチューナー クロマチックタイプ NS Micro...,Victor 22-5-65101-8 36-Piece Ready Emergency R...
1,Best laptop under $1000,"Bottle Cutter & Glass Cutter Kit, for Cutting ...",InSinkErator PRO750 with Cord Pro Series 3/4 H...,"LETMY LED Tactical Flashlight, Ultra Bright 20...","OEMTOOLS 24221 Torque Limiting Extension Set, ...",amFilm Protector Pantalla (2 Piezas) para Nint...,amFilm Tempered Glass Screen Protector for Nin...,Yodoit for iPhone 6s Plus Screen Replacement T...,バックトゥザフューチャー グッズ ホバーボード おもちゃ 玩具 ＳＦ 映画 ＢＴＴＦ ＢＴＦ...,Officially Licensed Back to the Future 1:1 Sca...,...,"Gruper Yoga Mat Non Slip, Eco Friendly Fitness...",2枚入り「amFilm 」Nintendo Switch 用 ガラスフィルム 強化ガラス 保...,Yodoit iPhone6s Plus フロントパネル 画面 液晶 取り付け簡単化 (ホー...,"TECCPO Cordless Drill, Drill Driver Set with 2...",【Caseology】 AirPods Pro ケース カバー 耐久性 AirPodsPro...,Caseology Vault for Apple Airpods Pro Case (20...,Miniafinador de pinza PW-CT-12 NS de Planet Waves,D'Addario Guitar Tuner - Micro Headstock Tuner...,D'Addario ダダリオ ヘッドストックチューナー クロマチックタイプ NS Micro...,Victor 22-5-65101-8 36-Piece Ready Emergency R...
2,Nike running shoes for women,"Bottle Cutter & Glass Cutter Kit, for Cutting ...",InSinkErator PRO750 with Cord Pro Series 3/4 H...,"LETMY LED Tactical Flashlight, Ultra Bright 20...","OEMTOOLS 24221 Torque Limiting Extension Set, ...",amFilm Protector Pantalla (2 Piezas) para Nint...,amFilm Tempered Glass Screen Protector for Nin...,Yodoit for iPhone 6s Plus Screen Replacement T...,バックトゥザフューチャー グッズ ホバーボード おもちゃ 玩具 ＳＦ 映画 ＢＴＴＦ ＢＴＦ...,Officially Licensed Back to the Future 1:1 Sca...,...,"Gruper Yoga Mat Non Slip, Eco Friendly Fitness...",2枚入り「amFilm 」Nintendo Switch 用 ガラスフィルム 強化ガラス 保...,Yodoit iPhone6s Plus フロントパネル 画面 液晶 取り付け簡単化 (ホー...,"TECCPO Cordless Drill, Drill Driver Set with 2...",【Caseology】 AirPods Pro ケース カバー 耐久性 AirPodsPro...,Caseology Vault for Apple Airpods Pro Case (20...,Miniafinador de pinza PW-CT-12 NS de Planet Waves,D'Addario Guitar Tuner - Micro Headstock Tuner...,D'Addario ダダリオ ヘッドストックチューナー クロマチックタイプ NS Micro...,Victor 22-5-65101-8 36-Piece Ready Emergency R...
3,iPhone 13 reviews,"Bottle Cutter & Glass Cutter Kit, for Cutting ...",InSinkErator PRO750 with Cord Pro Series 3/4 H...,"LETMY LED Tactical Flashlight, Ultra Bright 20...","OEMTOOLS 24221 Torque Limiting Extension Set, ...",amFilm Protector Pantalla (2 Piezas) para Nint...,amFilm Tempered Glass Screen Protector for Nin...,Yodoit for iPhone 6s Plus Screen Replacement T...,バックトゥザフューチャー グッズ ホバーボード おもちゃ 玩具 ＳＦ 映画 ＢＴＴＦ ＢＴＦ...,Officially Licensed Back to the Future 1:1 Sca...,...,"Gruper Yoga Mat Non Slip, Eco Friendly Fitness...",2枚入り「amFilm 」Nintendo Switch 用 ガラスフィルム 強化ガラス 保...,Yodoit iPhone6s Plus フロントパネル 画面 液晶 取り付け簡単化 (ホー...,"TECCPO Cordless Drill, Drill Driver Set with 2...",【Caseology】 AirP